In [ ]:
# %load_ext tensorboard


In [1]:
import sys
import datetime

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

sys.path.append("..")

from utils import paths

from sklearn.model_selection import train_test_split

pd.set_option("display.max_columns", None)

tf.config.list_physical_devices()


2023-07-05 16:23:24.703830: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 16:23:24.772920: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-05 16:23:24.790723: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-05 16:23:25.159547: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
data = pd.read_csv(paths.DATA_PATH + "/COVID19MX_sample.csv")
data.sample(10)


,SEXO,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,DEFUNCION,INTUBADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,CLASIFICACION_FINAL,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
32336,1,2,2022-06-24,2022-06-22,1,2.0,1.0,84.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,NaN,7,109.0,0.0,2.0
14362,2,2,2022-01-06,2022-01-04,1,2.0,1.0,60.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,3,109.0,0.0,2.0
16963,2,2,2022-07-23,2022-07-15,1,2.0,1.0,87.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,3,109.0,0.0,2.0
64950,2,1,2022-02-25,2022-02-23,0,3.0,2.0,11.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,3.0
25107,1,1,2022-06-12,2022-06-11,0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7,109.0,0.0,3.0
28890,2,1,2022-01-05,2022-01-04,0,3.0,2.0,20.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,3.0
13117,1,2,2022-03-19,2022-03-19,1,2.0,1.0,80.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,7,109.0,0.0,2.0
57222,2,2,2023-01-26,2023-01-25,1,2.0,1.0,60.0,2.0,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,3,109.0,0.0,2.0
57202,1,2,2022-02-08,2022-02-07,1,2.0,1.0,69.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,2.0
45097,1,2,2022-12-31,2022-12-26,0,2.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,2.0


In [3]:
X = data.drop(columns=["DEFUNCION", "FECHA_INGRESO", "FECHA_SINTOMAS"]).to_numpy()
Y = data[["DEFUNCION"]].to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=557)

print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")


Train: (74766, 21)
Test: (18692, 21)


In [4]:
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=2, num_classes=2,
    directory="../autokeras_search", project_name=f"experiment_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

train_history = clf.fit(
    x=X_train, y=Y_train,
    epochs=10, validation_split=0.25,
    callbacks=[tensorboard_callback]
)


Trial 2 Complete [00h 00m 54s]

Best val_accuracy So Far: None
Total elapsed time: 00h 01m 49s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
2337/2337 [==============================] - ETA: 0s - loss: 0.0942 - accuracy: 0.9756

AttributeError: 'VocabWeightHandler' object has no attribute 'name'

In [ ]:
%tensorboard --logdir logs/fit


In [5]:
predicted_y = clf.predict(X_test)


NotFoundError: ../autokeras_search/experiment_20230705-162443/best_pipeline; No such file or directory

In [ ]:
predicted_y


In [ ]:
clf.evaluate(X_test, Y_test)


In [ ]:
model = clf.export_model()
model.save("../models/model_autokeras", save_format="tf")


In [ ]:
model.summary()


In [ ]:
tf.keras.utils.plot_model(model)
